In [156]:
import pandas as pd
import numpy as np

In [157]:
data_base = pd.read_excel('./DATA_preprocess_EDA_base/big_merge_V2_meteo_SAT.xlsx').drop('Unnamed: 0', axis=1)

In [158]:
data_base.head()

,PARCELLE,LAT,LON,ALT,PRODREG,HT_VEG,DATE,SLOPE25,ASPECT25,ORIENTATION,...,TAVE_AVG,TAVE,TAVE_GROWTH,PRCP_S_S,PRCP_G_S,NDVI,EVI,NDMI,NDWI,DSWI
0,51,47.781623,8.612822,715.918970,1,3,1984-04-10,58.407726,67.342415,E,...,NaN,NaN,NaN,0.000000,0.000000,0.3779,0.0146,0.2264,-0.3325,0.4705
1,51,47.781623,8.612822,715.918970,1,3,1994-10-19,58.407726,67.342415,E,...,8.6567,9.017907,13.881706,115.666797,50.258061,0.4992,0.0160,0.2705,-0.4653,0.6865
2,51,47.781623,8.612822,715.918970,1,3,2011-11-09,58.407726,67.342415,E,...,8.8129,4.521227,14.696609,118.682917,55.673864,-0.1255,-0.0138,0.0107,0.1121,-0.0810
3,51,47.781623,8.612822,715.918970,1,3,2005-11-01,58.407726,67.342415,E,...,8.7947,3.070213,14.416862,122.371409,54.540621,0.6923,0.0272,0.2294,-0.6305,0.9579
4,384,47.736528,8.625154,563.829759,1,2,1995-07-10,55.683254,253.354935,O,...,8.3280,19.689020,13.741095,123.236130,55.141756,0.5552,0.0149,0.2017,-0.4864,0.6996


PREPROCESSING _ Code base for models temporal predictions

Ici, features engineering (création de nouvelles features à partir de la liste connues):

In [160]:
# Exemple, formule sécheresse...

### Choix TARGET et Features :

Target :

In [162]:
TARGET = ['TAUX_COUV_RAJ'] #exemple

Features :

In [163]:
# --- PAST --- 
# Attention : on peut, logiquement, inclure la TARGET... (connue dans le passé)
cat_strict = ['TAUX_COUV_RAJ','PRODREG','UNIT_VEG_GROS'] #exemple
cat_ord_miss = ['TAILLE_PPL', 'MELANGE', 'HT_VEG'] #exemple
numerics = ['ALT','SLOPE25','QUAL_STATION','AGE_PPL','PERF_CROI', 'PRCP', 'TAVE_AVG',	'TAVE', 'TAVE_GROWTH', 'PRCP_S_S', 'PRCP_G_S', 'NDVI', 'EVI', 'NDMI', 'NDWI', 'DSWI'] #exemple

# --- FUTURE KNOWN ---
# Attention : logiquement les features, potentiellement connues dans le futur ci-dessous sont aussi répertoriées ci-dessus dans le passé
add_cat_IFN_stable = []
add_cat_ord_IFN_stable = []
add_IFN_numerics_stable = ['ALT'] #exemple
# données potentiellement connues car stables par parcelles :
#['LAT', 'LON', 'ALT', 'PRODREG', 'HT_VEG', 'SLOPE25', 'ASPECT25', 'ORIENTATION', 'PERF_CROI', 'QUAL_STATION', 'UNIT_VEG_FINE',
# 'UNIT_VEG_GROS', 'PROCESS_SILVA', 'PERI_CHENAUX', 'PERI_COULEES','PERI_AVALANCH', 'PERI_CHUTES', 'ETAGE', 'ENDOMMAGEMENT','NB_DEGAT_ARBRE']
# + données extrapolées grâce à des technologies prometteuses (satellites ?)....

add_meteo_known = ['PRCP', 'TAVE_AVG',	'TAVE', 'TAVE_GROWTH', 'PRCP_S_S',	'PRCP_G_S']

add_SAT_known = ['NDVI', 'EVI', 'NDMI', 'NDWI', 'DSWI']

In [164]:
features_past = numerics + cat_strict + cat_ord_miss
features_future = add_cat_IFN_stable + add_cat_ord_IFN_stable + add_IFN_numerics_stable + add_meteo_known + add_SAT_known

In [165]:
data_LFI1 = data_base.loc[data_base['LFI']=='LFI1',:].sort_values('PARCELLE')
data_LFI2 = data_base.loc[data_base['LFI']=='LFI2',:].sort_values('PARCELLE')
data_LFI3 = data_base.loc[data_base['LFI']=='LFI3',:].sort_values('PARCELLE')
data_LFI4 = data_base.loc[data_base['LFI']=='LFI4',:].sort_values('PARCELLE')

In [166]:
future_feat_names = []
add_cat_strict_feat_names = []
add_cat_ord_feat_names = []
add_numerics_feat_names = []

for cat in features_future:
    lfi2_list = data_LFI2[cat].to_list()
    lfi3_list = data_LFI3[cat].to_list()
    lfi4_list = data_LFI4[cat].to_list()
    data_LFI1[cat + "_f"] = lfi2_list
    data_LFI2[cat + "_f"] = lfi2_list
    data_LFI3[cat + "_f"] = lfi2_list
    future_feat_names.append(cat + '_f')
    if cat in add_cat_ord_IFN_stable:
        add_cat_ord_feat_names.append(cat + '_f')
    elif cat in add_cat_IFN_stable:
        add_cat_strict_feat_names.append(cat + '_f')
    else:
        add_numerics_feat_names.append(cat + '_f')


In [167]:
data_red = pd.concat([data_LFI1, data_LFI2, data_LFI3], axis=0)[features_past + future_feat_names]
Y = pd.concat([data_LFI2[TARGET], data_LFI3[TARGET], data_LFI4[TARGET]], axis = 0)

In [168]:
data_red.head()

,ALT,SLOPE25,QUAL_STATION,AGE_PPL,PERF_CROI,PRCP,TAVE_AVG,TAVE,TAVE_GROWTH,PRCP_S_S,...,TAVE_AVG_f,TAVE_f,TAVE_GROWTH_f,PRCP_S_S_f,PRCP_G_S_f,NDVI_f,EVI_f,NDMI_f,NDWI_f,DSWI_f
0,715.918970,58.407726,4,85.0,4676,NaN,NaN,NaN,NaN,0.0,...,8.6567,9.017907,13.881706,115.666797,50.258061,0.4992,0.0160,0.2705,-0.4653,0.6865
5,563.829759,55.683254,3,140.0,3402,NaN,NaN,NaN,NaN,0.0,...,8.3280,19.689020,13.741095,123.236130,55.141756,0.5552,0.0149,0.2017,-0.4864,0.6996
9,564.885846,43.496788,4,80.0,5617,NaN,NaN,NaN,NaN,0.0,...,9.1600,12.259857,14.464487,110.778326,51.717751,0.5633,0.0153,0.1727,-0.5002,0.6852
14,563.551602,29.557123,3,2.0,4258,NaN,NaN,NaN,NaN,0.0,...,8.9985,11.576915,14.352682,111.841829,52.584194,0.6059,0.0197,0.2269,-0.5588,0.8384
19,539.769096,53.450974,3,130.0,4330,NaN,NaN,NaN,NaN,0.0,...,9.2676,12.310067,14.590373,108.512739,50.565995,0.6012,0.0184,0.2029,-0.5486,0.7902


In [169]:
Y.head()

,TAUX_COUV_RAJ
1,2
4,6
8,6
13,6
16,5


Traitement des données catégorielles ordonnées en numériques (gestion des "-1" éventuels) :

In [170]:
feats_cat_ord = cat_ord_miss + add_cat_ord_feat_names
for cat in feats_cat_ord:
  data_red[cat] = data_red[cat].apply(lambda v : v if v!=-1 else np.nan)

LISTE DES FEATURES NUMERIQUES ET CATEGORIELLES EN VUE DU PREPROCESSING DE MODELE :

In [171]:
numerics_features = numerics + feats_cat_ord + add_numerics_feat_names
feats_cat_strict = cat_strict + add_cat_strict_feat_names